<a href="https://colab.research.google.com/github/igorq937/PSPD-Colab_Spark-Kafka-Elasticsearch-Kibana/blob/main/Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Laboratório extra sobre Spark/Kafka (com API Gráfica)**

# Inicializações para o laboratório Google Colab

Instalando a biblioteca que permite baixar conteúdos do Google Drive

In [1]:
!pip --quiet install gdown
import gdown

Baixando pasta do Google Drive com configurações do Kafka Connect

In [2]:
url = 'https://drive.google.com/drive/folders/1GTbtftPQ_7j0QjNsQY9f2r23wABmgRXG?usp=drive_link'
gdown.download_folder(url)

Retrieving folder list


Processing file 1pHtGb3e3i08M41RV0WQY5TAFgfZhXsUq connect.properties
Processing file 11SJhUS4aKEs4qEBLhp_bbpwa_SBFvoOa connect.properties.bkp
Processing file 1qJh-9wxwCaXkNHmTv31mjeFcA-2pm3ag elasticsearch.properties
Processing file 1kU8Eh1P95GjTdz6fLbqdYi4aaB9q8ME8 elasticsearch.properties.bkp
Building directory structure completed


Retrieving folder list completed
Building directory structure
Downloading...
From: https://drive.google.com/uc?id=1pHtGb3e3i08M41RV0WQY5TAFgfZhXsUq
To: /content/kafka_config/connect.properties
100%|██████████| 577/577 [00:00<00:00, 1.54MB/s]
Downloading...
From: https://drive.google.com/uc?id=11SJhUS4aKEs4qEBLhp_bbpwa_SBFvoOa
To: /content/kafka_config/connect.properties.bkp
100%|██████████| 577/577 [00:00<00:00, 1.16MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qJh-9wxwCaXkNHmTv31mjeFcA-2pm3ag
To: /content/kafka_config/elasticsearch.properties
100%|██████████| 469/469 [00:00<00:00, 891kB/s]
Downloading...
From: https://drive.google.com/uc?id=1kU8Eh1P95GjTdz6fLbqdYi4aaB9q8ME8
To: /content/kafka_config/elasticsearch.properties.bkp
100%|██████████| 468/468 [00:00<00:00, 868kB/s]
Download completed


['/content/kafka_config/connect.properties',
 '/content/kafka_config/connect.properties.bkp',
 '/content/kafka_config/elasticsearch.properties',
 '/content/kafka_config/elasticsearch.properties.bkp']

# Instalação/configuração do Spark - versão cluster

Definindo as variáveis de ambiente do Spark (e do Hadoop?)

In [3]:
import os

# Variáveis gerais
os.environ['JAVA_HOME']="/usr/lib/jvm/java-11-openjdk-amd64" # readlink -f /usr/bin/javac
os.environ['BASHRC_PATH']= "/root/.bashrc"

# Variáveis específicas do Spark
os.environ['SPARK_INSTALL_DIR']="/content"
os.environ['SPARK_HOME']="/content/spark"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'

Baixando e descompactando o Hadoop

In [4]:
%%bash
wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz -P $SPARK_INSTALL_DIR
tar -xvzf $SPARK_INSTALL_DIR/spark-3.5.0-bin-hadoop3.tgz -C $SPARK_INSTALL_DIR
mv $SPARK_INSTALL_DIR/spark-3.5.0-bin-hadoop3 $SPARK_INSTALL_DIR/spark
rm $SPARK_INSTALL_DIR/spark-3.5.0-bin-hadoop3.tgz

spark-3.5.0-bin-hadoop3/
spark-3.5.0-bin-hadoop3/kubernetes/
spark-3.5.0-bin-hadoop3/kubernetes/tests/
spark-3.5.0-bin-hadoop3/kubernetes/tests/pyfiles.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/decommissioning.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/autoscale.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/python_executable_check.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/worker_memory_check.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/py_container_checks.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/decommissioning_cleanup.py
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/decom.sh
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/bindings/
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/bindings/R/
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/bindings/R/Dockerfile
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/bindings/python/
spark-3.5.0-bin-hadoop3/kubern

Iniciando os processos NameNode e DataNode, daemons do HDFS

In [5]:
!$SPARK_HOME/sbin/start-master.sh --host localhost --port 7077

starting org.apache.spark.deploy.master.Master, logging to /content/spark/logs/spark--org.apache.spark.deploy.master.Master-1-f303db030412.out


Iniciando os processos relativos ao gerenciador de recursos YARN

In [6]:
!$SPARK_HOME/sbin/start-worker.sh spark://localhost:7077

starting org.apache.spark.deploy.worker.Worker, logging to /content/spark/logs/spark--org.apache.spark.deploy.worker.Worker-1-f303db030412.out


# Instalação/configuração do Elastic Search - versão OSS

Baixando e descompactando o Elastic Search

In [7]:
%%bash
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.10.2-linux-x86_64.tar.gz
tar -xzf elasticsearch-oss-7.10.2-linux-x86_64.tar.gz
rm elasticsearch-oss-7.10.2-linux-x86_64.tar.gz
mv elasticsearch-7.10.2 elasticsearch
sudo chown -R daemon:daemon elasticsearch

Ativando o daemon do Elastic Search

In [8]:
%%bash --bg
sudo -H -u daemon elasticsearch/bin/elasticsearch

# Instalação/configuração do Kibana - versão OSS

Baixando e descompactando o Kibana

In [9]:
%%bash
wget -q https://artifacts.elastic.co/downloads/kibana/kibana-oss-7.10.2-linux-x86_64.tar.gz
tar -xzf kibana-oss-7.10.2-linux-x86_64.tar.gz
rm kibana-oss-7.10.2-linux-x86_64.tar.gz
mv kibana-7.10.2-linux-x86_64 kibana
sudo chown -R daemon:daemon kibana

Ativando o daemon do Kibana

In [10]:
%%bash --bg
sudo -H -u daemon kibana/bin/kibana > kibana/kibana.log 2>&1

# Instalação/configuração do kafka

Baixando e descompactando o kafka

In [11]:
%%bash
wget -q https://dlcdn.apache.org/kafka/3.5.1/kafka_2.13-3.5.1.tgz
tar xvf kafka_2.13-3.5.1.tgz
rm kafka_2.13-3.5.1.tgz
mv kafka_2.13-3.5.1 kafka

kafka_2.13-3.5.1/
kafka_2.13-3.5.1/LICENSE
kafka_2.13-3.5.1/NOTICE
kafka_2.13-3.5.1/bin/
kafka_2.13-3.5.1/bin/connect-distributed.sh
kafka_2.13-3.5.1/bin/connect-mirror-maker.sh
kafka_2.13-3.5.1/bin/connect-standalone.sh
kafka_2.13-3.5.1/bin/kafka-acls.sh
kafka_2.13-3.5.1/bin/kafka-broker-api-versions.sh
kafka_2.13-3.5.1/bin/kafka-cluster.sh
kafka_2.13-3.5.1/bin/kafka-configs.sh
kafka_2.13-3.5.1/bin/kafka-console-consumer.sh
kafka_2.13-3.5.1/bin/kafka-console-producer.sh
kafka_2.13-3.5.1/bin/kafka-consumer-groups.sh
kafka_2.13-3.5.1/bin/kafka-consumer-perf-test.sh
kafka_2.13-3.5.1/bin/kafka-delegation-tokens.sh
kafka_2.13-3.5.1/bin/kafka-delete-records.sh
kafka_2.13-3.5.1/bin/kafka-dump-log.sh
kafka_2.13-3.5.1/bin/kafka-e2e-latency.sh
kafka_2.13-3.5.1/bin/kafka-features.sh
kafka_2.13-3.5.1/bin/kafka-get-offsets.sh
kafka_2.13-3.5.1/bin/kafka-jmx.sh
kafka_2.13-3.5.1/bin/kafka-leader-election.sh
kafka_2.13-3.5.1/bin/kafka-log-dirs.sh
kafka_2.13-3.5.1/bin/kafka-metadata-quorum.sh
kafka_2.1

Ativando os daemons do Kafka

In [12]:
%%bash
./kafka/bin/zookeeper-server-start.sh -daemon ./kafka/config/zookeeper.properties
./kafka/bin/kafka-server-start.sh -daemon ./kafka/config/server.properties

Criando canais Kafka

In [13]:
%%bash
./kafka/bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic canalinput
./kafka/bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic canaloutput

Created topic canalinput.
Created topic canaloutput.


[2023-12-19 20:40:16,424] WARN [AdminClient clientId=adminclient-1] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available. (org.apache.kafka.clients.NetworkClient)
[2023-12-19 20:40:16,458] WARN [AdminClient clientId=adminclient-1] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available. (org.apache.kafka.clients.NetworkClient)
[2023-12-19 20:40:16,560] WARN [AdminClient clientId=adminclient-1] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available. (org.apache.kafka.clients.NetworkClient)
[2023-12-19 20:40:16,762] WARN [AdminClient clientId=adminclient-1] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available. (org.apache.kafka.clients.NetworkClient)
[2023-12-19 20:40:17,266] WARN [AdminClient clientId=adminclient-1] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broke

# Instalação/configuração do Kafka Connect Elastic Search

Baixando e descompactando o Kafka Connect Elastic Search

In [14]:
%%bash
wget -q https://d1i4a15mxbxib1.cloudfront.net/api/plugins/confluentinc/kafka-connect-elasticsearch/versions/11.2.4/confluentinc-kafka-connect-elasticsearch-11.2.4.zip
unzip confluentinc-kafka-connect-elasticsearch-11.2.4.zip
rm confluentinc-kafka-connect-elasticsearch-11.2.4.zip
mkdir -p kafka/plugins
mv confluentinc-kafka-connect-elasticsearch-11.2.4 kafka/plugins/confluentinc-kafka-connect-elasticsearch

Archive:  confluentinc-kafka-connect-elasticsearch-11.2.4.zip
   creating: confluentinc-kafka-connect-elasticsearch-11.2.4/doc/
  inflating: confluentinc-kafka-connect-elasticsearch-11.2.4/doc/version.txt  
   creating: confluentinc-kafka-connect-elasticsearch-11.2.4/doc/licenses/
  inflating: confluentinc-kafka-connect-elasticsearch-11.2.4/doc/notices/NOTICE-httpcore-nio-4.4.4.txt  
  inflating: confluentinc-kafka-connect-elasticsearch-11.2.4/doc/licenses/LICENSE-httpclient-4.5.1.txt  
  inflating: confluentinc-kafka-connect-elasticsearch-11.2.4/doc/licenses/LICENSE-httpcore-nio-4.4.4.txt  
  inflating: confluentinc-kafka-connect-elasticsearch-11.2.4/doc/licenses/LICENSE-httpasyncclient-4.1.1.txt  
  inflating: confluentinc-kafka-connect-elasticsearch-11.2.4/doc/licenses/LICENSE-httpcore-4.4.4.txt  
  inflating: confluentinc-kafka-connect-elasticsearch-11.2.4/assets/elasticsearch.jpg  
  inflating: confluentinc-kafka-connect-elasticsearch-11.2.4/lib/jackson-databind-2.15.2.jar  
  inf

Movendo arquivos de configuração do Kafka Connect

In [15]:
%%bash
mv kafka_config/*.properties kafka/config/
rm -r kafka_config/

Ativando os daemons do Kafka Connect

In [16]:
!kafka/bin/connect-standalone.sh -daemon kafka/config/connect.properties kafka/config/elasticsearch.properties

# Contabilizando as palavras do kafka no streaming

Instala pyspark

In [17]:
!pip install --quiet pyspark
from pyspark.sql import SparkSession

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Inicializa constantes para criação da sessão Spark

In [18]:
SPARK_MASTER="spark://localhost:7077"
APP_NAME="Lab Kafka Spark"

KAFKA_HOST="localhost:9092"
KAFKA_TOPIC_IN="canalinput"
KAFKA_TOPIC_OUT="canaloutput"

INTERVAL = "5 seconds"

Cria sessão Spark com PySpark com um processo de streaming de dados utilizando o Kafka como fonte e destino do dados.

In [19]:
spark = SparkSession.builder.master(SPARK_MASTER).appName(APP_NAME).getOrCreate()

In [20]:
messages = spark.readStream.format("kafka").option("kafka.bootstrap.servers", KAFKA_HOST) \
                .option("subscribe", KAFKA_TOPIC_IN) \
                .option("includeTimestamp", "true").load()

In [21]:
from pyspark.sql.functions import explode, split, col, upper, window, to_json, struct, lit, regexp_replace

In [22]:
words = messages \
            .select(
                explode(split(col("value"), "\s+")).alias("word"),
                messages.timestamp
        ).select (
            upper(col("word")).alias("word"),
            col("timestamp")
        )

In [23]:
wordCounts = words.withWatermark("timestamp", INTERVAL) \
                .groupBy(
                    window(words.timestamp, INTERVAL, INTERVAL), "word"
                ).count()

In [24]:
wordCountsJson = wordCounts.select(
    lit("1").alias("key"),
    to_json(struct("word", "count", 'window.start')).alias("value"))

In [25]:
wck = wordCountsJson.writeStream.outputMode("update").format("kafka").option("kafka.bootstrap.servers", KAFKA_HOST) \
        .option("topic", KAFKA_TOPIC_OUT).option("checkpointLocation", "/tmp/spark-kafka") \
        .trigger(processingTime=INTERVAL).start()

# Instalação/configuração da API de consumo de rede social

Instala o NLTK (Natural Language Toolkit)

In [26]:
!pip install --quiet nltk

Cria função para remoção de caracteres especiais, pontuações e stop words (com o NLTK)

In [27]:
import re, nltk
from nltk.corpus import stopwords
import string

nltk.download('stopwords')
nltk.download('punkt')

# Cria um conjunto de stop words
stop_words = set(stopwords.words('english'))

# Define uma função para remover stop words e pontuações de uma sentença
def remove_stop_words_and_punctuation(sentence):
    # Divide a sentença em palavras individuais
    words = nltk.word_tokenize(sentence)

    # Remove stop words e pontuações
    filtered_words = [word for word in words if word.lower() not in stop_words and bool(re.match('^\w+$', word))]

    # Junta as palavras filtradas de volta em uma sentença
    return ' '.join(filtered_words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Instala o kafka-python e PRAW (The Python Reddit API Wrapper)

In [28]:
!pip install --quiet kafka-python
!pip install --quiet praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 3.3 MB/s eta 0:00:00


Realiza 5 requisições de 100 tópicos no reddit r/news \
Para executar e necessário as secrets a seguir:

*   `redditClientId`
*   `redditClientSecret`
*   `redditUser` (Nome de usuário do reddit)

`redditClientId` e `redditClientSecret` podem ser obtidas cadastrando um app em "https://www.reddit.com/prefs/apps".



In [29]:
import praw
from copy import copy
from kafka import KafkaProducer
from google.colab import userdata

reddit = praw.Reddit(
  client_id=userdata.get('redditClientId'),
  client_secret=userdata.get('redditClientSecret'),
  user_agent='using praw by u/' + userdata.get('redditUser'),
  check_for_async=False,
)

subreddit = reddit.subreddit('geek')
hot_topics = subreddit.hot(limit=100)

# Configurar producer para o Kafka
producer = KafkaProducer(bootstrap_servers=KAFKA_HOST)

limit_per_request = 100
num_requests = 5

for index in range(num_requests):
  if index == 0:
    hot_topics = subreddit.hot(limit=limit_per_request)
  else:
    # Em iterações subsequentes, realizaz requisições de tópicos após o último tópico da iteração anterior
    hot_topics = subreddit.hot(limit=limit_per_request, params={'after': last_topic.name})

  # Obtem cópia do último tópico da lista de tópicos
  last_topic = list(copy(hot_topics))[-1]

  for topic in hot_topics:
    # Remover stopwords e pontuações do título
    value = remove_stop_words_and_punctuation(topic.title)
    print(value)

    # Enviar o valor processado para o tópico Kafka
    producer.send(KAFKA_TOPIC_IN, value=value.encode('utf-8'))

# Aguardar que as mensagens sejam enviadas
producer.flush()

# Fechar o produtor
producer.close()

sound black hole Recorded NASA Chandra Observatory
think custom mjolnir sign made
drawing used interpret quotes
Twitter search available registered users
geek internet craze Going friend house LAN party
Geek themed Dry Bar
Dapper fella reality warping powers
Time Magazine journalist challenged Marine running Senate game Magic Gathering lost
Figured guys would like garden sign wife made 40th birthday
Elon Musk says SUE Microsoft illegally using Twitter data row ads
best geek store USA Canada
Working tech trivia night friends thought share one Put Windows versions order release note Windows version removed covers
Super Mario Movie 6th Video Game Movie Ever 5 Days
miss old thinkgeek find best new ones
ChatGPT fooled generating old Windows keys illustrates broader problem AI
50 years day since first cell phone call
Apple product development team signatures seen inside shell Macintosh
living room 2 years ago years ago today Evolution nerd cave
need wireless device tell far away another devi

# Apresentação de resultados em um dashboard gráfico no Kibana/ElasticSearch

Instala pyngrok

In [30]:
!pip install --quiet pyngrok
from pyngrok import ngrok, conf

Cria tunelamento para acesso ao servidor do Kibana com Ngrok \
Necessário a secret `ngrokAuthToken` com o auth_token do Ngrok que pode ser visualizado no link abaixo: \
https://dashboard.ngrok.com/get-started/your-authtoken

In [31]:
port = 5601 # Porta padrão do Kibana

conf.get_default().auth_token = userdata.get('ngrokAuthToken')
public_url = ngrok.connect(port).public_url

print(" * Link de acesso ao Kibana \"{}\"".format(public_url, port))

 * Link de acesso ao Kibana "https://6123-35-230-47-39.ngrok-free.app"
